# TFG - Anàlisi de l’abandonament escolar prematur a Catalunya
## Tractament de Dades Any 2019
Jahziel Hidalgo Artigas



## Taula de continguts<a class="anchor" id="index"></a>
* [Imports](#imports)
* [Funcions](#funcions)
* [Càrrega Datasets](#datasets)
* [Comarques](#comarques)
* [Dades Formació](#formacio)
    * [Anàlisi per Categoría](#formacio)
    * [Formació d'Adults](#formacio_adults)
* [Dades Població](#poblacio)
   * [Població per Gènere](#poblacio_genere)
   * [Població per Edat](#poblacio_edat)
   * [Atur per Edat](#atur)
   * [Atur per Gènere](#atur_genere)
   * [Percentatge/Ratio Estudiants](#pct_estudiants)
   * [Població per Nacionalitat](#poblacio_nacionalitat)
* [Dades Socioeconòmiques](#dades_socioeconomiques)
    * [Repetidors](#repetidors)
    * [Total Repetidors per Comarca](#repetidors_comarca)
    * [Impacte de la Nacionalitat](#extrangers)
    * [Nivell Socioeconòmic (Renda Familiar)](#renta_familiar)
    * [Nivell Socioeconòmic (RFBD](#rfbd)


### Imports<a class="anchor" id="imports"></a>

In [321]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import unicodedata
import os
from unidecode import unidecode

### Funcions<a class="anchor" id="funcions"></a>

In [9]:
def clean_dataframe(df):
    # Netejar les columnes: treure els accents, substituir els espais per guions baixos i convertir a minúscules
    df.columns = [unidecode(col.replace(' ', '_').lower()) for col in df.columns]
    
    # Netejar les dades: treure els accents, convertir a minúscules i processar només les cadenes de text
    df = df.apply(lambda x: x.apply(lambda val: unidecode(str(val)).lower() if isinstance(val, str) else val) 
                  if x.dtype == 'object' else x, axis=0)
    
    return df

def filtrar_comarques(df, columna):
    comarques_valides = [
        'alt camp', 'alt emporda', 'alt penedes', 'alt urgell', 'alta ribagorca', 
        'anoia', 'aran', 'bages', 'baix camp', 'baix ebre', 'baix emporda', 
        'baix llobregat', 'baix penedes', 'barcelones', 'bergueda', 'cerdanya', 
        'conca de barbera', 'garraf', 'garrigues', 'garrotxa', 'girones', 'maresme', 
        'moianes', 'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira', 
        "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre", 'ripolles', 
        'segarra', 'segria', 'selva', 'solsones', 'tarragones', 'terra alta', 'urgell', 
        'valles occidental', 'valles oriental'
    ]
    
    # Filtrar les files que continguin valors en la columna dins de la llista de comarques vàlides
    df = df[df[columna].isin(comarques_valides)].reset_index(drop=True)
    
    return df


### Càrrega Datasets<a class="anchor" id="datasets"></a>

In [11]:
df_comarques = pd.read_csv('csv/2019/comarques.csv', delimiter=';', skiprows=3) # Filtar per Nivell = 'Comarca'
df_formacio_sexe = pd.read_csv('csv/2019/nivell_formacio_sexe_2019.csv', delimiter=';', skiprows=6) 
df_formacio_adults = pd.read_csv('csv/2019/formacio_adults_2019.csv', delimiter=';', skiprows=5)
df_poblacio_genere = pd.read_csv('csv/2019/poblacio_genere_2019.csv', delimiter=';')
df_poblacio_edat = pd.read_csv('csv/2019/poblacio_edat_2019.csv', delimiter=';')
df_poblacio_nacionalitat = pd.read_csv('csv/2019/poblacio_nacionalitat_2019.csv', delimiter=';', skiprows=6)
df_repetidors = pd.read_csv('csv/2019/alumnes_repetidors_2019.csv', delimiter=';')
df_renta_familiar = pd.read_csv('csv/2019/renta_familiar2019.csv', delimiter=';', skiprows=6)
df_rfbd = pd.read_csv('csv/2019/RFBD_2019.csv', delimiter=';', skiprows=6)
df_atur_edat = pd.read_csv('csv/2019/atur_edat_2019.csv', delimiter=';', skiprows=6)
df_atur_genere = pd.read_csv('csv/2019/atur_genere_2019.csv', delimiter=';', skiprows=6)

In [269]:
datasets = {
    'Comarques': df_comarques,
    'Formació': df_formacio,
    'Formació Adults': df_formacio_adults,
    'Població Sexe': df_poblacio_genere,
    'Població Edat': df_poblacio_edat,
    'Població Nacionalitat': df_poblacio_nacionalitat,
    'Repetidors': df_repetidors,
    'Renta Familiar': df_renta_familiar,
    'RFBD': df_rfbd
}

# Iterar sobre el diccionari per mostrar les primeres files de cada dataset
for name, df in datasets.items():
    print(f"Primeres files de {name}:")
    display(df.head()) 
    print("\n") 

Primeres files de Comarques:


,codi,nom
0,01,alt camp
1,02,alt emporda
2,03,alt penedes
3,04,alt urgell
4,05,alta ribagorca




Primeres files de Formació:


,comarca,analfabets,educacio_primaria_incompleta,educacio_primaria,primera_etapa_d'educacio_secundaria_i_similar,segona_etapa_d'educacio_secundaria_amb_orientacio_general,segona_etapa_d'educacio_secundaria_amb_orientacio_professional,ensenyament_de_grau_superior_de_formacio_professional_i_similars,graus_universitaris_de_fins_a_240_credits_i_similars,graus_universitaris_de_mes_de_240_credits_i_similars,master_universitari_i_similars,doctorat_universitari,total
0,Alt Camp,115,1545,4525,13570,3935,4200,3855,2815,2355,580,185,37680
1,Alt Empordà,375,6900,15795,42130,17755,11160,6685,8725,7610,1555,590,119275
2,Alt Penedès,205,2385,12235,28235,10410,11150,8580,7915,6780,1480,555,89920
3,Alt Urgell,45,795,2085,5380,2630,1790,1470,1555,1325,355,115,17550
4,Alta Ribagorça,10,145,390,910,495,320,285,350,335,60,25,3325




Primeres files de Formació Adults:


...

,codi,comarca,sexe,repetixen
0,01,alt camp,dona,71
1,01,alt camp,home,115
2,02,alt emporda,dona,254
3,02,alt emporda,home,438
4,03,alt penedes,dona,169




Primeres files de Renta Familiar:


,comarca,rfdb_(miles_de_euros),rfdb_por_habitante_(euros),rfdb_por_habitante_(index_catalunya=100_por_habitante)
0,Alt Camp,720501,16266,"88,1"
1,Alt Empordà,2034758,14802,"80,2"
2,Alt Penedès,1839020,17023,"92,2"
3,Alt Urgell,293967,14569,"78,9"
4,Alta Ribagorça,61538,16124,"87,3"




Primeres files de RFBD:


,comarca,recursos._remuneracion_asalariados,recursos._excedente_bruto_explotacion,recursos._prestaciones_sociales,usos._cotizaciones_sociales,usos._impuestos
0,Alt Camp,565929,186515,206768,171532,91718
1,Alt Empordà,1551611,680144,484650,486006,309468
2,Alt Penedès,1527182,496000,510175,452437,259924
3,Alt Urgell,187736,95819,85046,60167,34504
4,Alta Ribagorça,41587,24789,15395,13962,7562


### Comarques<a class="anchor" id="comarques"></a>

L'estudi es farà per comarques, així que necessitem obtenir el codi de les comarques, el qual ens servirà per agrupar el conjunt final.

In [15]:
# Primerament, netegem el dataset per assegurar-nos que no hi hagi problemes amb els noms de les columnes, accents, etc.
df_comarques = clean_dataframe(df_comarques)

In [17]:
df_comarques.head()

,nivell,codi,nom
0,comarca,1,alt camp
1,municipi,430017,aiguamurcia
2,municipi,430056,alcover
3,municipi,430108,alio
4,municipi,430347,brafim


In [19]:
df_comarques.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nivell  990 non-null    object
 1   codi    990 non-null    int64 
 2   nom     990 non-null    object
dtypes: int64(1), object(2)
memory usage: 23.3+ KB


In [21]:
# Filtrarem les dades per seleccionar només aquelles files on el 'nivell' sigui 'comarca',
# i només seleccionarem les columnes 'codi' i 'nom' que seran necessàries per a l'anàlisi posterior.
df_comarques = df_comarques[df_comarques['nivell'] == 'comarca'][['codi', 'nom']].reset_index(drop=True)
# Convertim la columna 'codi' a format de dos dígits
df_comarques['codi'] = df_comarques['codi'].astype(str).str.zfill(2)
df_comarques.head()

,codi,nom
0,01,alt camp
1,02,alt emporda
2,03,alt penedes
3,04,alt urgell
4,05,alta ribagorca


In [23]:
# Afegim manualment el codi d'Aran (39)
df_comarques.loc[len(df_comarques)] = ['39', 'aran']
df_comarques = df_comarques.sort_values(by='codi').reset_index(drop=True)
df_comarques.tail()

,codi,nom
38,39,aran
39,40,valles occidental
40,41,valles oriental
41,42,moianes
42,43,llucanes


In [25]:
df_comarques['codi'] = df_comarques['codi'].astype(str)
df_comarques.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   codi    43 non-null     object
 1   nom     43 non-null     object
dtypes: object(2)
memory usage: 820.0+ bytes


In [27]:
# Comprovem els noms únics de les comarques per assegurar-nos que no hi ha comarques duplicades
df_comarques['nom'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'bages', 'baix camp', 'baix ebre',
       'baix emporda', 'baix llobregat', 'baix penedes', 'barcelones',
       'bergueda', 'cerdanya', 'conca de barbera', 'garraf', 'garrigues',
       'garrotxa', 'girones', 'maresme', 'montsia', 'noguera', 'osona',
       'pallars jussa', 'pallars sobira', "pla d'urgell",
       "pla de l'estany", 'priorat', "ribera d'ebre", 'ripolles',
       'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'aran', 'valles occidental',
       'valles oriental', 'moianes', 'llucanes'], dtype=object)

In [29]:
# Imprimim el nombre de comarques úniques que tenim en el dataset
print(f"Hi han {len(df_comarques['nom'].unique())} comarques")

Hi han 43 comarques


In [31]:
# Crear el diccionari de codis (comarca -> codi)
comarques_dict = df_comarques.set_index('nom')['codi'].to_dict()
comarques_dict

{'alt camp': '01',
 'alt emporda': '02',
 'alt penedes': '03',
 'alt urgell': '04',
 'alta ribagorca': '05',
 'anoia': '06',
 'bages': '07',
 'baix camp': '08',
 'baix ebre': '09',
 'baix emporda': '10',
 'baix llobregat': '11',
 'baix penedes': '12',
 'barcelones': '13',
 'bergueda': '14',
 'cerdanya': '15',
 'conca de barbera': '16',
 'garraf': '17',
 'garrigues': '18',
 'garrotxa': '19',
 'girones': '20',
 'maresme': '21',
 'montsia': '22',
 'noguera': '23',
 'osona': '24',
 'pallars jussa': '25',
 'pallars sobira': '26',
 "pla d'urgell": '27',
 "pla de l'estany": '28',
 'priorat': '29',
 "ribera d'ebre": '30',
 'ripolles': '31',
 'segarra': '32',
 'segria': '33',
 'selva': '34',
 'solsones': '35',
 'tarragones': '36',
 'terra alta': '37',
 'urgell': '38',
 'aran': '39',
 'valles occidental': '40',
 'valles oriental': '41',
 'moianes': '42',
 'llucanes': '43'}

### Dades Formació<a class="anchor" id="formacio"></a>

#### Anàlisi per Categoría<a class="anchor" id="formacio"></a>

In [299]:
df_formacio = pd.read_csv('csv/2019/nivell_formacio_2019.csv', delimiter=';', skiprows=6) 

In [301]:
# Netejar el dataset per assegurar que les columnes siguin coherents
df_formacio_clean = clean_dataframe(df_formacio)
df_formacio_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 13 columns):
 #   Column                                                            Non-Null Count  Dtype 
---  ------                                                            --------------  ----- 
 0   comarca                                                           55 non-null     object
 1   analfabets                                                        55 non-null     int64 
 2   educacio_primaria_incompleta                                      55 non-null     int64 
 3   educacio_primaria                                                 55 non-null     int64 
 4   primera_etapa_d'educacio_secundaria_i_similar                     55 non-null     int64 
 5   segona_etapa_d'educacio_secundaria_amb_orientacio_general         55 non-null     int64 
 6   segona_etapa_d'educacio_secundaria_amb_orientacio_professional    55 non-null     int64 
 7   ensenyament_de_grau_superior_de_formacio_profe

In [303]:
# Filtrar per comarques
df_formacio_clean = filtrar_comarques(df_formacio_clean, 'comarca')

In [305]:
# Comprovem els canvis realitzats
df_formacio_clean.info()
display(df_formacio_clean.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 13 columns):
 #   Column                                                            Non-Null Count  Dtype 
---  ------                                                            --------------  ----- 
 0   comarca                                                           42 non-null     object
 1   analfabets                                                        42 non-null     int64 
 2   educacio_primaria_incompleta                                      42 non-null     int64 
 3   educacio_primaria                                                 42 non-null     int64 
 4   primera_etapa_d'educacio_secundaria_i_similar                     42 non-null     int64 
 5   segona_etapa_d'educacio_secundaria_amb_orientacio_general         42 non-null     int64 
 6   segona_etapa_d'educacio_secundaria_amb_orientacio_professional    42 non-null     int64 
 7   ensenyament_de_grau_superior_de_formacio_profe

,comarca,analfabets,educacio_primaria_incompleta,educacio_primaria,primera_etapa_d'educacio_secundaria_i_similar,segona_etapa_d'educacio_secundaria_amb_orientacio_general,segona_etapa_d'educacio_secundaria_amb_orientacio_professional,ensenyament_de_grau_superior_de_formacio_professional_i_similars,graus_universitaris_de_fins_a_240_credits_i_similars,graus_universitaris_de_mes_de_240_credits_i_similars,master_universitari_i_similars,doctorat_universitari,total
0,alt camp,115,1545,4525,13570,3935,4200,3855,2815,2355,580,185,37680
1,alt emporda,375,6900,15795,42130,17755,11160,6685,8725,7610,1555,590,119275
2,alt penedes,205,2385,12235,28235,10410,11150,8580,7915,6780,1480,555,89920
3,alt urgell,45,795,2085,5380,2630,1790,1470,1555,1325,355,115,17550
4,alta ribagorca,10,145,390,910,495,320,285,350,335,60,25,3325
5,anoia,365,3425,12295,35190,11370,12245,9890,7820,6320,1415,435,100775
6,aran,15,140,705,2565,1505,1030,865,955,760,145,40,8725
7,bages,415,5035,20170,50360,17195,16445,13160,13405,11195,2545,805,150735
8,baix camp,1305,6775,18080,52000,20550,16770,14275,13135,12155,3230,1060,159335
9,baix ebre,165,3565,9685,23470,8055,7070,4985,4585,3770,1210,270,66830


In [307]:
# Comprovem els canvis realitzats
df_formacio_clean.info()
display(df_formacio_clean.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 13 columns):
 #   Column                                                            Non-Null Count  Dtype 
---  ------                                                            --------------  ----- 
 0   comarca                                                           42 non-null     object
 1   analfabets                                                        42 non-null     int64 
 2   educacio_primaria_incompleta                                      42 non-null     int64 
 3   educacio_primaria                                                 42 non-null     int64 
 4   primera_etapa_d'educacio_secundaria_i_similar                     42 non-null     int64 
 5   segona_etapa_d'educacio_secundaria_amb_orientacio_general         42 non-null     int64 
 6   segona_etapa_d'educacio_secundaria_amb_orientacio_professional    42 non-null     int64 
 7   ensenyament_de_grau_superior_de_formacio_profe

,comarca,analfabets,educacio_primaria_incompleta,educacio_primaria,primera_etapa_d'educacio_secundaria_i_similar,segona_etapa_d'educacio_secundaria_amb_orientacio_general,segona_etapa_d'educacio_secundaria_amb_orientacio_professional,ensenyament_de_grau_superior_de_formacio_professional_i_similars,graus_universitaris_de_fins_a_240_credits_i_similars,graus_universitaris_de_mes_de_240_credits_i_similars,master_universitari_i_similars,doctorat_universitari,total
0,alt camp,115,1545,4525,13570,3935,4200,3855,2815,2355,580,185,37680
1,alt emporda,375,6900,15795,42130,17755,11160,6685,8725,7610,1555,590,119275
2,alt penedes,205,2385,12235,28235,10410,11150,8580,7915,6780,1480,555,89920
3,alt urgell,45,795,2085,5380,2630,1790,1470,1555,1325,355,115,17550
4,alta ribagorca,10,145,390,910,495,320,285,350,335,60,25,3325
5,anoia,365,3425,12295,35190,11370,12245,9890,7820,6320,1415,435,100775
6,aran,15,140,705,2565,1505,1030,865,955,760,145,40,8725
7,bages,415,5035,20170,50360,17195,16445,13160,13405,11195,2545,805,150735
8,baix camp,1305,6775,18080,52000,20550,16770,14275,13135,12155,3230,1060,159335
9,baix ebre,165,3565,9685,23470,8055,7070,4985,4585,3770,1210,270,66830


In [309]:
# Definim el mapeig de les categories educatives
cat = {
    'analfabetisme_total': ['analfabets'],
    'educacio primaria incompleta_total': ['educacio_primaria_incompleta'],
    'educacio primaria_total': ['educacio_primaria'],
    'educacio secundaria_total': ['primera_etapa_d\'educacio_secundaria_i_similar', 
                            'segona_etapa_d\'educacio_secundaria_amb_orientacio_general',
                            'segona_etapa_d\'educacio_secundaria_amb_orientacio_professional'],
    'formacio professional_total': ['ensenyament_de_grau_superior_de_formacio_professional_i_similars'],
    'graus universitaris_total': ['graus_universitaris_de_fins_a_240_credits_i_similars', 
                            'graus_universitaris_de_mes_de_240_credits_i_similars'],
    'masters universitaris_total': ['master_universitari_i_similars'],
    'doctorat universitari_total': ['doctorat_universitari']
}

# Creem un nou DataFrame per emmagatzemar els resultats
df_grouped = pd.DataFrame()

# Iterem sobre les categories i sumem les columnes corresponents
for categoria, columnas in cat.items():
    # Sumar las columnas que corresponden a la categoría
    df_grouped[categoria] = df_formacio_clean[columnas].sum(axis=1)

# Afegim una nova columna amb el total per comarca, sumant les columnes anteriors
df_grouped['total'] = df_grouped.sum(axis=1)  # Sumar las columnas para obtener el total por comarca

# Mostrar el DataFrame resultante con las nuevas columnas de categorías
df_grouped.head()

,analfabetisme_total,educacio primaria incompleta_total,educacio primaria_total,educacio secundaria_total,formacio professional_total,graus universitaris_total,masters universitaris_total,doctorat universitari_total,total
0,115,1545,4525,21705,3855,5170,580,185,37680
1,375,6900,15795,71045,6685,16335,1555,590,119280
2,205,2385,12235,49795,8580,14695,1480,555,89930
3,45,795,2085,9800,1470,2880,355,115,17545
4,10,145,390,1725,285,685,60,25,3325


In [311]:
# Merge entre df_grouped i df_formacio_clean per afegir la columna 'comarca' de df_formacio_clean
df_formacio_clean = df_grouped.merge(df_formacio_clean[['comarca']], left_index=True, right_index=True, how='left')
df_formacio_clean.head()

,analfabetisme_total,educacio primaria incompleta_total,educacio primaria_total,educacio secundaria_total,formacio professional_total,graus universitaris_total,masters universitaris_total,doctorat universitari_total,total,comarca
0,115,1545,4525,21705,3855,5170,580,185,37680,alt camp
1,375,6900,15795,71045,6685,16335,1555,590,119280,alt emporda
2,205,2385,12235,49795,8580,14695,1480,555,89930,alt penedes
3,45,795,2085,9800,1470,2880,355,115,17545,alt urgell
4,10,145,390,1725,285,685,60,25,3325,alta ribagorca


In [313]:
# Reordenem les columnes per posar 'comarca' al principi
df_formacio_clean = df_formacio_clean[['comarca'] + [col for col in df_grouped.columns if col != 'comarca']]
df_formacio_clean.head()

,comarca,analfabetisme_total,educacio primaria incompleta_total,educacio primaria_total,educacio secundaria_total,formacio professional_total,graus universitaris_total,masters universitaris_total,doctorat universitari_total,total
0,alt camp,115,1545,4525,21705,3855,5170,580,185,37680
1,alt emporda,375,6900,15795,71045,6685,16335,1555,590,119280
2,alt penedes,205,2385,12235,49795,8580,14695,1480,555,89930
3,alt urgell,45,795,2085,9800,1470,2880,355,115,17545
4,alta ribagorca,10,145,390,1725,285,685,60,25,3325


In [315]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_formacio_clean.insert(0, 'codi', df_formacio_clean['comarca'].map(comarques_dict))
df_formacio_clean = df_formacio_clean.sort_values(by='codi').reset_index(drop=True)
df_formacio_clean.head()

,codi,comarca,analfabetisme_total,educacio primaria incompleta_total,educacio primaria_total,educacio secundaria_total,formacio professional_total,graus universitaris_total,masters universitaris_total,doctorat universitari_total,total
0,01,alt camp,115,1545,4525,21705,3855,5170,580,185,37680
1,02,alt emporda,375,6900,15795,71045,6685,16335,1555,590,119280
2,03,alt penedes,205,2385,12235,49795,8580,14695,1480,555,89930
3,04,alt urgell,45,795,2085,9800,1470,2880,355,115,17545
4,05,alta ribagorca,10,145,390,1725,285,685,60,25,3325


In [317]:
df_formacio_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   codi                                42 non-null     object
 1   comarca                             42 non-null     object
 2   analfabetisme_total                 42 non-null     int64 
 3   educacio primaria incompleta_total  42 non-null     int64 
 4   educacio primaria_total             42 non-null     int64 
 5   educacio secundaria_total           42 non-null     int64 
 6   formacio professional_total         42 non-null     int64 
 7   graus universitaris_total           42 non-null     int64 
 8   masters universitaris_total         42 non-null     int64 
 9   doctorat universitari_total         42 non-null     int64 
 10  total                               42 non-null     int64 
dtypes: int64(9), object(2)
memory usage: 3.7+ KB


In [319]:
df_formacio_clean.to_csv('clean_data/2019/df_formacio.csv', index=False)

#### Formació d'Adults<a class="anchor" id="formacio_adults"></a>

In [79]:
df_formacio_adults = pd.read_csv('csv/2019/formacio_adults_2019.csv', delimiter=';', skiprows=5)

In [81]:
df_formacio_adults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   comarca                    43 non-null     object
 1   Centres                    43 non-null     int64 
 2   Personal docent            43 non-null     int64 
 3   Alumnes (homes). 16-24     43 non-null     int64 
 4   Alumnes (homes). 25-64     43 non-null     int64 
 5   Alumnes (homes). 65 i més  43 non-null     int64 
 6   Alumnes (homes). Total     43 non-null     int64 
 7   Alumnes (dones). 16-24     43 non-null     int64 
 8   Alumnes (dones). 25-64     43 non-null     int64 
 9   Alumnes (dones). 65 i més  43 non-null     int64 
 10  Alumnes (dones). Total     43 non-null     int64 
 11  Alumnes (total). 16-24     43 non-null     int64 
 12  Alumnes (total). 25-64     43 non-null     int64 
 13  Alumnes (total). 65 i més  43 non-null     int64 
 14  Alumnes (tot

In [83]:
df_formacio_adults.head(10)

,comarca,Centres,Personal docent,Alumnes (homes). 16-24,Alumnes (homes). 25-64,Alumnes (homes). 65 i més,Alumnes (homes). Total,Alumnes (dones). 16-24,Alumnes (dones). 25-64,Alumnes (dones). 65 i més,Alumnes (dones). Total,Alumnes (total). 16-24,Alumnes (total). 25-64,Alumnes (total). 65 i més,Alumnes (total). Total
0,Alt Camp,1,7,24,45,11,80,20,125,38,183,44,170,49,263
1,Alt Empordà,4,33,183,557,18,758,84,288,26,398,267,845,44,1156
2,Alt Penedès,3,16,58,62,16,136,36,103,29,168,94,165,45,304
3,Alt Urgell,1,6,23,80,27,130,24,188,30,242,47,268,57,372
4,Alta Ribagorça,1,1,1,2,0,3,5,13,0,18,6,15,0,21
5,Anoia,1,4,34,42,4,80,21,35,1,57,55,77,5,137
6,Aran,1,3,7,57,3,67,6,129,8,143,13,186,11,210
7,Bages,3,34,95,427,33,555,64,362,61,487,159,789,94,1042
8,Baix Camp,4,39,207,344,52,603,174,645,62,881,381,989,114,1484
9,Baix Ebre,2,13,61,75,25,161,52,222,42,316,113,297,67,477


In [85]:
# Netejar el dataset per assegurar que les columnes siguin coherents
df_formacio_adults_clean = clean_dataframe(df_formacio_adults)
df_formacio_adults_clean.head()

,comarca,centres,personal_docent,alumnes_(homes)._16-24,alumnes_(homes)._25-64,alumnes_(homes)._65_i_mes,alumnes_(homes)._total,alumnes_(dones)._16-24,alumnes_(dones)._25-64,alumnes_(dones)._65_i_mes,alumnes_(dones)._total,alumnes_(total)._16-24,alumnes_(total)._25-64,alumnes_(total)._65_i_mes,alumnes_(total)._total
0,alt camp,1,7,24,45,11,80,20,125,38,183,44,170,49,263
1,alt emporda,4,33,183,557,18,758,84,288,26,398,267,845,44,1156
2,alt penedes,3,16,58,62,16,136,36,103,29,168,94,165,45,304
3,alt urgell,1,6,23,80,27,130,24,188,30,242,47,268,57,372
4,alta ribagorca,1,1,1,2,0,3,5,13,0,18,6,15,0,21


In [87]:
# Filtrar per comarques
df_formacio_adults_clean = filtrar_comarques(df_formacio_adults_clean, 'comarca')
df_formacio_adults_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   comarca                    42 non-null     object
 1   centres                    42 non-null     int64 
 2   personal_docent            42 non-null     int64 
 3   alumnes_(homes)._16-24     42 non-null     int64 
 4   alumnes_(homes)._25-64     42 non-null     int64 
 5   alumnes_(homes)._65_i_mes  42 non-null     int64 
 6   alumnes_(homes)._total     42 non-null     int64 
 7   alumnes_(dones)._16-24     42 non-null     int64 
 8   alumnes_(dones)._25-64     42 non-null     int64 
 9   alumnes_(dones)._65_i_mes  42 non-null     int64 
 10  alumnes_(dones)._total     42 non-null     int64 
 11  alumnes_(total)._16-24     42 non-null     int64 
 12  alumnes_(total)._25-64     42 non-null     int64 
 13  alumnes_(total)._65_i_mes  42 non-null     int64 
 14  alumnes_(tot

In [89]:
# Suma d'alumnes majors de 25 per a homes, dones i el total
df_formacio_adults_clean.loc[:, 'alumnes_homes_+25_anys'] = df_formacio_adults_clean['alumnes_(homes)._25-64'] + df_formacio_adults['alumnes_(homes)._65_i_mes']
df_formacio_adults_clean.loc[:, 'alumnes_dones_+25_anys'] = df_formacio_adults_clean['alumnes_(dones)._25-64'] + df_formacio_adults['alumnes_(dones)._65_i_mes']
df_formacio_adults_clean.loc[:, 'alumnes_total_+25_anys'] = df_formacio_adults_clean['alumnes_(total)._25-64'] + df_formacio_adults['alumnes_(total)._65_i_mes']


# Convertir les columnes +25 anys a tipus int
df_formacio_adults_clean[['alumnes_homes_+25_anys', 'alumnes_dones_+25_anys', 'alumnes_total_+25_anys']] = (
    df_formacio_adults_clean[['alumnes_homes_+25_anys', 'alumnes_dones_+25_anys', 'alumnes_total_+25_anys']].astype(int)
)

# Seleccionar i renombrar columnes
df_formacio_adults_clean = df_formacio_adults_clean[['comarca',
                                         'alumnes_(homes)._16-24',
                                         'alumnes_(dones)._16-24',
                                         'alumnes_(total)._16-24',
                                         'alumnes_homes_+25_anys',
                                         'alumnes_dones_+25_anys',
                                         'alumnes_total_+25_anys']].rename(columns={
    'alumnes_(homes)._16-24': 'alumnes_homes_16_24_anys',
    'alumnes_(dones)._16-24': 'alumnes_dones_16_24_anys',
    'alumnes_(total)._16-24': 'alumnes_total_16_24_anys'
})
# Dataset resultant
df_formacio_adults_clean.head()


,comarca,alumnes_homes_16_24_anys,alumnes_dones_16_24_anys,alumnes_total_16_24_anys,alumnes_homes_+25_anys,alumnes_dones_+25_anys,alumnes_total_+25_anys
0,alt camp,24,20,44,56,163,219
1,alt emporda,183,84,267,575,314,889
2,alt penedes,58,36,94,78,132,210
3,alt urgell,23,24,47,107,218,325
4,alta ribagorca,1,5,6,2,13,15


In [91]:
df_formacio_adults_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   comarca                   42 non-null     object
 1   alumnes_homes_16_24_anys  42 non-null     int64 
 2   alumnes_dones_16_24_anys  42 non-null     int64 
 3   alumnes_total_16_24_anys  42 non-null     int64 
 4   alumnes_homes_+25_anys    42 non-null     int32 
 5   alumnes_dones_+25_anys    42 non-null     int32 
 6   alumnes_total_+25_anys    42 non-null     int32 
dtypes: int32(3), int64(3), object(1)
memory usage: 1.9+ KB


In [93]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_formacio_adults_clean.insert(0, 'codi', df_formacio_adults_clean['comarca'].map(comarques_dict))
df_formacio_adults_clean = df_formacio_adults_clean.sort_values(by='codi').reset_index(drop=True)
df_formacio_adults_clean.head()

,codi,comarca,alumnes_homes_16_24_anys,alumnes_dones_16_24_anys,alumnes_total_16_24_anys,alumnes_homes_+25_anys,alumnes_dones_+25_anys,alumnes_total_+25_anys
0,01,alt camp,24,20,44,56,163,219
1,02,alt emporda,183,84,267,575,314,889
2,03,alt penedes,58,36,94,78,132,210
3,04,alt urgell,23,24,47,107,218,325
4,05,alta ribagorca,1,5,6,2,13,15


In [95]:
df_formacio_adults_clean.to_csv('clean_data/2019/df_formacio_adults.csv', index=False)

### Dades Població<a class="anchor" id="poblacio"></a>

#### Població per Gènere<a class="anchor" id="poblacio_genere"></a>

In [323]:
df_poblacio_genere = pd.read_csv('csv/2019/poblacio_genere_2019.csv', delimiter=';') 

In [325]:
df_poblacio_genere.head()

,comarca,dones,homes,total
0,Alt Camp,21829,22466,44295
1,Alt Empordà,69885,70426,140311
2,Alt Penedès,54145,53945,108090
3,Alt Urgell,10120,10175,20295
4,Alta Ribagorça,1846,1958,3804


In [327]:
# Netejar el dataset per assegurar que les columnes siguin coherents
df_poblacio_genere_clean = clean_dataframe(df_poblacio_genere)
df_poblacio_genere_clean.head()

,comarca,dones,homes,total
0,alt camp,21829,22466,44295
1,alt emporda,69885,70426,140311
2,alt penedes,54145,53945,108090
3,alt urgell,10120,10175,20295
4,alta ribagorca,1846,1958,3804


In [329]:
df_poblacio_genere_clean['comarca'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental',
       'catalunya', 'metropolita', 'comarques gironines',
       'camp de tarragona', "terres de l'ebre", 'ponent',
       'comarques centrals', 'alt pirineu i aran', 'penedes', 'barcelona',
       'girona', 'lleida', 'tarragona'], dtype=object)

In [331]:
# Filtrem per comarques
df_poblacio_genere_clean = filtrar_comarques(df_poblacio_genere_clean, 'comarca')

In [333]:
df_poblacio_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comarca  42 non-null     object
 1   dones    42 non-null     int64 
 2   homes    42 non-null     int64 
 3   total    42 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.4+ KB


In [335]:
df_poblacio_genere_clean.head()

,comarca,dones,homes,total
0,alt camp,21829,22466,44295
1,alt emporda,69885,70426,140311
2,alt penedes,54145,53945,108090
3,alt urgell,10120,10175,20295
4,alta ribagorca,1846,1958,3804


In [337]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_genere_clean.insert(0, 'codi', df_poblacio_genere_clean['comarca'].map(comarques_dict))
df_poblacio_genere_clean = df_poblacio_genere_clean.sort_values(by='codi').reset_index(drop=True)
df_poblacio_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   codi     42 non-null     object
 1   comarca  42 non-null     object
 2   dones    42 non-null     int64 
 3   homes    42 non-null     int64 
 4   total    42 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 1.8+ KB


In [339]:
df_poblacio_genere_clean.head()

,codi,comarca,dones,homes,total
0,01,alt camp,21829,22466,44295
1,02,alt emporda,69885,70426,140311
2,03,alt penedes,54145,53945,108090
3,04,alt urgell,10120,10175,20295
4,05,alta ribagorca,1846,1958,3804


In [341]:
df_poblacio_genere_clean.to_csv('clean_data/2019/df_poblacio_genere.csv', index=False)

#### Població per Edat<a class="anchor" id="poblacio_edat"></a>

In [343]:
df_poblacio_edat = pd.read_csv('csv/2019/poblacio_edat_2019.csv', delimiter=';')

In [345]:
df_poblacio_edat.head()

,comarca,poblacio_total_16_a_24_anys,poblacio_homes_16_a_24_anys,poblacio_dones_16_a_24_anys
0,Alt Camp,4063,2118,1945
1,Alt Empordà,12663,6535,6128
2,Alt Penedès,9605,4865,4740
3,Alt Urgell,1720,910,810
4,Alta Ribagorça,292,141,151


In [347]:
df_poblacio_edat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   comarca                      55 non-null     object
 1   poblacio_total_16_a_24_anys  55 non-null     int64 
 2   poblacio_homes_16_a_24_anys  55 non-null     int64 
 3   poblacio_dones_16_a_24_anys  55 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.8+ KB


In [349]:
# Netejar el DataFrame 'df_poblacio_edat' utilitzant la funció 'clean_dataframe'
df_poblacio_edat_clean = clean_dataframe(df_poblacio_edat)
df_poblacio_edat_clean.head()

,comarca,poblacio_total_16_a_24_anys,poblacio_homes_16_a_24_anys,poblacio_dones_16_a_24_anys
0,alt camp,4063,2118,1945
1,alt emporda,12663,6535,6128
2,alt penedes,9605,4865,4740
3,alt urgell,1720,910,810
4,alta ribagorca,292,141,151


In [351]:
# Filtrar les comarques utilitzant la funció 'filtrar_comarques'
df_poblacio_edat_clean = filtrar_comarques(df_poblacio_edat_clean, 'comarca')
df_poblacio_edat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   comarca                      42 non-null     object
 1   poblacio_total_16_a_24_anys  42 non-null     int64 
 2   poblacio_homes_16_a_24_anys  42 non-null     int64 
 3   poblacio_dones_16_a_24_anys  42 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.4+ KB


In [353]:
df_poblacio_edat_clean.head()

,comarca,poblacio_total_16_a_24_anys,poblacio_homes_16_a_24_anys,poblacio_dones_16_a_24_anys
0,alt camp,4063,2118,1945
1,alt emporda,12663,6535,6128
2,alt penedes,9605,4865,4740
3,alt urgell,1720,910,810
4,alta ribagorca,292,141,151


In [355]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_edat_clean.insert(0, 'codi', df_poblacio_edat_clean['comarca'].map(comarques_dict))
df_poblacio_edat_clean = df_poblacio_edat_clean.sort_values(by='codi').reset_index(drop=True)
df_poblacio_edat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   codi                         42 non-null     object
 1   comarca                      42 non-null     object
 2   poblacio_total_16_a_24_anys  42 non-null     int64 
 3   poblacio_homes_16_a_24_anys  42 non-null     int64 
 4   poblacio_dones_16_a_24_anys  42 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 1.8+ KB


In [357]:
df_poblacio_edat_clean.to_csv('clean_data/2019/df_poblacio_edat.csv', index=False)

#### Població Activa (16-24 anys)<a class="anchor" id="poblacio_activa"></a>

In [ ]:
df_poblacio_activa = pd.read_csv('csv/2019/poblacio_activa_2019.csv', delimiter=';', skiprows=6)

In [157]:
df_poblacio_activa.head()

NameError: name 'df_poblacio_activa' is not defined

In [ ]:
# Netejar el DataFrame 'df_poblacio_activa' utilitzant la funció 'clean_dataframe'
df_poblacio_activa_clean = clean_dataframe(df_poblacio_activa)
df_poblacio_activa_clean.head()

In [ ]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
df_poblacio_activa_clean['comarca'].unique()

In [ ]:
# Filtrar les dades del DataFrame per obtenir només les comarques necesaries
df_poblacio_activa_clean = filtrar_comarques(df_poblacio_activa_clean, 'comarca')
df_poblacio_activa_clean.info()

In [ ]:
# Crear una nova columna 'poblacio_activa_16_a_24_anys' sumant les columnes 'de_16_a_19_anys' i 'de_20_a_24_anys'
df_poblacio_activa_clean.loc[:, 'poblacio_activa_16_a_24_anys'] = (
    df_poblacio_activa_clean['de_16_a_19_anys'] + df_poblacio_activa_clean['de_20_a_24_anys']
)
df_poblacio_activa_clean.head()

In [ ]:
# Seleccionar només les columnes 'comarca' i 'poblacio_activa_16_a_24_anys'
df_poblacio_activa = df_poblacio_activa_clean[['comarca','poblacio_activa_16_a_24_anys']]
df_poblacio_activa.head()

In [ ]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_activa.insert(0, 'codi', df_poblacio_activa['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_poblacio_activa = df_poblacio_activa.sort_values(by='codi').reset_index(drop=True)
df_poblacio_activa.head()

In [ ]:
# Fusionar els DataFrames 'df_poblacio_activa' i 'df_poblacio_genere_clean' utilitzant la columna comuna 'codi'
df_poblacio_activa_merged = df_poblacio_activa.merge(df_poblacio_genere_clean[['codi', 'total']], on='codi', how='left')
df_poblacio_activa_merged.head()

In [ ]:
# Percentatge de població activa (16-24 anys) respecte al total
df_poblacio_activa_merged['pct_activa'] = (df_poblacio_activa_merged['poblacio_activa_16_a_24_anys'] / df_poblacio_activa_merged['total'] * 100).round(2)

# Ràtio de població activa (16-24 anys) respecte al total
df_poblacio_activa_merged['ratio_activa'] = (df_poblacio_activa_merged['poblacio_activa_16_a_24_anys'] / df_poblacio_activa_merged['total'])
df_poblacio_activa_merged.head()

In [ ]:
df_poblacio_activa_merged.to_csv('clean_data/2019/df_poblacio_activa.csv', index=False)

#### Població Inactiva<a class="anchor" id="poblacio_inactiva"></a>

In [ ]:
df_poblacio_inactiva.info()

In [ ]:
df_poblacio_inactiva.head()

In [ ]:
# Netejar el DataFrame 'df_poblacio_inactiva' amb la funció 'clean_dataframe'
df_poblacio_inactiva_clean = clean_dataframe(df_poblacio_inactiva)
df_poblacio_inactiva_clean.head()

In [ ]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
df_poblacio_inactiva_clean['comarca'].unique()

In [ ]:
# Filtrar les comarques en el DataFrame netejat
df_poblacio_inactiva_clean = filtrar_comarques(df_poblacio_inactiva_clean, 'comarca')
# Obtenir el nombre total de comarques úniques després del filtratge
len(df_poblacio_inactiva_clean['comarca'].unique())

In [ ]:
df_poblacio_inactiva_clean['comarca'].unique()

In [ ]:
df_poblacio_inactiva_clean.info()

In [ ]:
df_poblacio_inactiva_clean.head()

In [ ]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_inactiva_clean.insert(0, 'codi', df_poblacio_inactiva_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_poblacio_inactiva_clean = df_poblacio_inactiva_clean.sort_values(by='codi').reset_index(drop=True)
df_poblacio_inactiva_clean.head()

In [ ]:
# Fusionar els DataFrames 'df_poblacio_inactiva_clean' i 'df_poblacio_genere_clean' utilitzant la columna comuna 'codi'
df_poblacio_inactiva_merged = df_poblacio_inactiva_clean.merge(df_poblacio_genere_clean[['codi', 'total']], on='codi', how='left')
df_poblacio_inactiva_merged.head()

In [ ]:
# Percentatge de població inactiva respecte al total
df_poblacio_inactiva_merged['pct_atur'] = (df_poblacio_inactiva_clean['poblacio_amb_una_altra_situacio_d\'inactivitat'] / df_poblacio_inactiva_merged['total'] * 100).round(2)

# Ràtio de població inactiva respecte al total
df_poblacio_inactiva_merged['ratio_atur'] = (df_poblacio_inactiva_clean['poblacio_amb_una_altra_situacio_d\'inactivitat'] / df_poblacio_inactiva_merged['total'])
df_poblacio_inactiva_merged.head()

In [ ]:
# Canviar el nom de la columna 'total' per 'total_poblacio'
df_poblacio_inactiva_merged.rename(columns={'total': 'total_poblacio'}, inplace=True)
df_poblacio_inactiva_merged.info()

In [ ]:
df_poblacio_inactiva_merged['total_poblacio'].sum()

In [ ]:
df_poblacio_inactiva_merged.to_csv('clean_data/2019/df_poblacio_inactiva.csv', index=False)

#### Atur per Edat<a class="anchor" id="atur"></a>

In [ ]:
df_atur_edat = pd.read_csv('csv/2019/atur_edat_2019.csv', delimiter=';', skiprows=6)

In [159]:
df_atur_edat.head()

,comarca,16-24,25+,Total
0,Alt Camp,199.5,2370.0,2569.5
1,Alt Empordà,588.0,6581.0,7169.0
2,Alt Penedès,324.1,4952.6,5276.7
3,Alt Urgell,58.9,710.8,769.7
4,Alta Ribagorça,5.6,80.9,86.5


In [161]:
df_atur_edat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   comarca  55 non-null     object 
 1   16-24    55 non-null     float64
 2   25+      55 non-null     float64
 3   Total    55 non-null     float64
dtypes: float64(3), object(1)
memory usage: 1.8+ KB


In [163]:
# Netejar el DataFrame 'df_atur_edat' amb la funció 'clean_dataframe'
df_atur_edat_clean = clean_dataframe(df_atur_edat)
df_atur_edat_clean.head()

,comarca,16-24,25+,total
0,alt camp,199.5,2370.0,2569.5
1,alt emporda,588.0,6581.0,7169.0
2,alt penedes,324.1,4952.6,5276.7
3,alt urgell,58.9,710.8,769.7
4,alta ribagorca,5.6,80.9,86.5


In [165]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
df_atur_edat_clean['comarca'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental',
       'cataluna', 'metropolita', 'comarques gironines',
       'camp de tarragona', "terres de l'ebre", 'ponent',
       'comarques centrals', 'alt pirineu i aran', 'penedes', 'barcelona',
       'girona', 'lleida', 'tarragona'], dtype=object)

In [167]:
# Filtrar les comarques en el DataFrame netejat
df_atur_edat_clean = filtrar_comarques(df_atur_edat_clean, 'comarca')
# Obtenir el nombre total de comarques úniques després del filtratge
len(df_atur_edat_clean['comarca'].unique())

42

In [169]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_atur_edat_clean.insert(0, 'codi', df_atur_edat_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_atur_edat_clean = df_atur_edat_clean.sort_values(by='codi').reset_index(drop=True)
df_atur_edat_clean.head()

,codi,comarca,16-24,25+,total
0,01,alt camp,199.5,2370.0,2569.5
1,02,alt emporda,588.0,6581.0,7169.0
2,03,alt penedes,324.1,4952.6,5276.7
3,04,alt urgell,58.9,710.8,769.7
4,05,alta ribagorca,5.6,80.9,86.5


In [171]:
df_atur_edat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   codi     42 non-null     object 
 1   comarca  42 non-null     object 
 2   16-24    42 non-null     float64
 3   25+      42 non-null     float64
 4   total    42 non-null     float64
dtypes: float64(3), object(2)
memory usage: 1.8+ KB


In [173]:
# Identificar les columnes de tipus float al dataset
columnes_float = df_atur_edat_clean.select_dtypes(include=['float']).columns

# Convertir les columnes de float a int
df_atur_edat_clean[columnes_float] = (
    np.ceil(df_atur_edat_clean[columnes_float]).astype(int)
)
# Verificar el resultat
df_atur_edat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   codi     42 non-null     object
 1   comarca  42 non-null     object
 2   16-24    42 non-null     int32 
 3   25+      42 non-null     int32 
 4   total    42 non-null     int32 
dtypes: int32(3), object(2)
memory usage: 1.3+ KB


In [175]:
df_atur_edat_clean.head()

,codi,comarca,16-24,25+,total
0,01,alt camp,200,2370,2570
1,02,alt emporda,588,6581,7169
2,03,alt penedes,325,4953,5277
3,04,alt urgell,59,711,770
4,05,alta ribagorca,6,81,87


In [177]:
df_atur_edat_clean.to_csv('clean_data/2019/df_atur_edat.csv', index=False)

#### Atur per Gènere<a class="anchor" id="atur_genere"></a>

In [359]:
df_atur_genere = pd.read_csv('csv/2019/atur_genere_2019.csv', delimiter=';', skiprows=6)

In [361]:
df_atur_genere.head()

,comarca,Total_homes,Total_dones,Total
0,Alt Camp,1094.1,1475.3,2569.4
1,Alt Empordà,3172.2,3996.8,7169.0
2,Alt Penedès,2173.3,3103.4,5276.7
3,Alt Urgell,351.5,418.2,769.7
4,Alta Ribagorça,42.8,43.7,86.5


In [363]:
df_atur_genere.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   comarca      55 non-null     object 
 1   Total_homes  55 non-null     float64
 2   Total_dones  55 non-null     float64
 3   Total        55 non-null     float64
dtypes: float64(3), object(1)
memory usage: 1.8+ KB


In [365]:
# Netejar el DataFrame 'df_atur_genere' amb la funció 'clean_dataframe'
df_atur_genere_clean = clean_dataframe(df_atur_genere)
df_atur_genere_clean.head()

,comarca,total_homes,total_dones,total
0,alt camp,1094.1,1475.3,2569.4
1,alt emporda,3172.2,3996.8,7169.0
2,alt penedes,2173.3,3103.4,5276.7
3,alt urgell,351.5,418.2,769.7
4,alta ribagorca,42.8,43.7,86.5


In [367]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
df_atur_genere_clean['comarca'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental',
       'cataluna', 'metropolita', 'comarques gironines',
       'camp de tarragona', "terres de l'ebre", 'ponent',
       'comarques centrals', 'alt pirineu i aran', 'penedes', 'barcelona',
       'girona', 'lleida', 'tarragona'], dtype=object)

In [369]:
# Filtrar les comarques en el DataFrame netejat
df_atur_genere_clean = filtrar_comarques(df_atur_genere_clean, 'comarca')
# Obtenir el nombre total de comarques úniques després del filtratge
len(df_atur_genere_clean['comarca'].unique())

42

In [371]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_atur_genere_clean.insert(0, 'codi', df_atur_genere_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_atur_genere_clean = df_atur_genere_clean.sort_values(by='codi').reset_index(drop=True)
df_atur_genere_clean.head()

,codi,comarca,total_homes,total_dones,total
0,01,alt camp,1094.1,1475.3,2569.4
1,02,alt emporda,3172.2,3996.8,7169.0
2,03,alt penedes,2173.3,3103.4,5276.7
3,04,alt urgell,351.5,418.2,769.7
4,05,alta ribagorca,42.8,43.7,86.5


In [373]:
df_atur_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   codi         42 non-null     object 
 1   comarca      42 non-null     object 
 2   total_homes  42 non-null     float64
 3   total_dones  42 non-null     float64
 4   total        42 non-null     float64
dtypes: float64(3), object(2)
memory usage: 1.8+ KB


In [375]:
# Identificar les columnes de tipus float al dataset
columnes_float = df_atur_genere_clean.select_dtypes(include=['float']).columns

# Convertir les columnes de float a int
df_atur_genere_clean[columnes_float] = (
    np.ceil(df_atur_genere_clean[columnes_float]).astype(int)
)
# Verificar el resultat
df_atur_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   codi         42 non-null     object
 1   comarca      42 non-null     object
 2   total_homes  42 non-null     int32 
 3   total_dones  42 non-null     int32 
 4   total        42 non-null     int32 
dtypes: int32(3), object(2)
memory usage: 1.3+ KB


In [377]:
df_atur_genere_clean.head()

,codi,comarca,total_homes,total_dones,total
0,01,alt camp,1095,1476,2570
1,02,alt emporda,3173,3997,7169
2,03,alt penedes,2174,3104,5277
3,04,alt urgell,352,419,770
4,05,alta ribagorca,43,44,87


In [379]:
df_atur_genere_clean.to_csv('clean_data/2019/df_atur_genere.csv', index=False)

#### Percentatge/Ratio Estudiants<a class="anchor" id="pct_estudiants"></a>

In [ ]:
df_poblacio_inactiva_merged.info()

In [ ]:
# Crear un nou DataFrame amb les columnes 'codi', 'comarca', 'poblacio_estudiant' i 'total_poblacio'
df_pct_ratio_estudiants = df_poblacio_inactiva_merged[['codi', 'comarca', 'poblacio_estudiant', 'total_poblacio']].copy()

# Calcular el percentatge d'estudiants respecte al total de població
df_pct_ratio_estudiants['pct_estudiants'] = (df_poblacio_inactiva_merged['poblacio_estudiant'] / df_poblacio_inactiva_merged['total_poblacio'] * 100).round(2)

# Calcular la ràtio d'estudiants respecte al total de població
df_pct_ratio_estudiants['ratio_estudiants'] = (df_poblacio_inactiva_merged['poblacio_estudiant'] / df_poblacio_inactiva_merged['total_poblacio'])
df_pct_ratio_estudiants.head()

In [ ]:
df_pct_ratio_estudiants.info()

In [ ]:
df_pct_ratio_estudiants.to_csv('clean_data/2019/df_estudiants_perc_ratio.csv', index=False)

#### Població per Nacionalitat<a class="anchor" id="poblacio_nacionalitat"></a>

In [203]:
df_poblacio_nacionalitat = pd.read_csv('csv/2019/poblacio_nacionalitat_2019.csv', delimiter=';', skiprows=6)

In [205]:
df_poblacio_nacionalitat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   comarca                     55 non-null     object
 1   Espanyola                   55 non-null     int64 
 2   Resta UE                    55 non-null     int64 
 3   Resta d'Europa              55 non-null     int64 
 4   Àfrica                      55 non-null     int64 
 5   Amèrica del Nord i Central  55 non-null     int64 
 6   Amèrica del Sud             55 non-null     int64 
 7   Àsia i Oceania              55 non-null     int64 
 8   Total                       55 non-null     int64 
dtypes: int64(8), object(1)
memory usage: 4.0+ KB


In [207]:
df_poblacio_nacionalitat.head()

,comarca,Espanyola,Resta UE,Resta d'Europa,Àfrica,Amèrica del Nord i Central,Amèrica del Sud,Àsia i Oceania,Total
0,Alt Camp,38831,1222,467,2767,191,569,249,44296
1,Alt Empordà,107134,11865,2054,13157,2167,3892,1070,141339
2,Alt Penedès,96742,1868,652,6127,573,1725,724,108411
3,Alt Urgell,17822,1044,212,345,150,504,100,20177
4,Alta Ribagorça,3365,171,61,121,19,59,6,3802


In [209]:
# Netejar el DataFrame 'df_poblacio_nacionalitat' amb la funció 'clean_dataframe'
df_poblacio_nacionalitat_clean = clean_dataframe(df_poblacio_nacionalitat)
# Filtrar les comarques en el DataFrame netejat
df_poblacio_nacionalitat_clean = filtrar_comarques(df_poblacio_nacionalitat_clean, 'comarca')
df_poblacio_nacionalitat_clean.head()

,comarca,espanyola,resta_ue,resta_d'europa,africa,america_del_nord_i_central,america_del_sud,asia_i_oceania,total
0,alt camp,38831,1222,467,2767,191,569,249,44296
1,alt emporda,107134,11865,2054,13157,2167,3892,1070,141339
2,alt penedes,96742,1868,652,6127,573,1725,724,108411
3,alt urgell,17822,1044,212,345,150,504,100,20177
4,alta ribagorca,3365,171,61,121,19,59,6,3802


In [211]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_nacionalitat_clean.insert(0, 'codi', df_poblacio_nacionalitat_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_poblacio_nacionalitat_clean = df_poblacio_nacionalitat_clean.sort_values(by='codi').reset_index(drop=True)
df_poblacio_nacionalitat_clean.head()

,codi,comarca,espanyola,resta_ue,resta_d'europa,africa,america_del_nord_i_central,america_del_sud,asia_i_oceania,total
0,01,alt camp,38831,1222,467,2767,191,569,249,44296
1,02,alt emporda,107134,11865,2054,13157,2167,3892,1070,141339
2,03,alt penedes,96742,1868,652,6127,573,1725,724,108411
3,04,alt urgell,17822,1044,212,345,150,504,100,20177
4,05,alta ribagorca,3365,171,61,121,19,59,6,3802


In [213]:
# Llista de nacionalitats
nacionalitats = ['espanyola', 'resta_ue', 'resta_d\'europa', 'africa', 'america_del_nord_i_central', 'america_del_sud', 'asia_i_oceania']

# Calcular el percentatge i la ràtio per a cada nacionalitat
for nacionalitat in nacionalitats:
    # Percentatge respecte al total per comarca
    df_poblacio_nacionalitat_clean[f'{nacionalitat}_pct'] = (
        df_poblacio_nacionalitat_clean[nacionalitat] / df_poblacio_nacionalitat_clean['total'] * 100
    ).round(2)
    
    # Ràtio respecte al total per comarca
    df_poblacio_nacionalitat_clean[f'{nacionalitat}_ratio'] = (
        df_poblacio_nacionalitat_clean[nacionalitat] / df_poblacio_nacionalitat_clean['total']
    )

# Mostrar les primeres files del DataFrame resultant
df_poblacio_nacionalitat_clean.head()

,codi,comarca,espanyola,resta_ue,resta_d'europa,africa,america_del_nord_i_central,america_del_sud,asia_i_oceania,total,...,resta_d'europa_pct,resta_d'europa_ratio,africa_pct,africa_ratio,america_del_nord_i_central_pct,america_del_nord_i_central_ratio,america_del_sud_pct,america_del_sud_ratio,asia_i_oceania_pct,asia_i_oceania_ratio
0,01,alt camp,38831,1222,467,2767,191,569,249,44296,...,1.05,0.010543,6.25,0.062466,0.43,0.004312,1.28,0.012845,0.56,0.005621
1,02,alt emporda,107134,11865,2054,13157,2167,3892,1070,141339,...,1.45,0.014532,9.31,0.093088,1.53,0.015332,2.75,0.027537,0.76,0.007570
2,03,alt penedes,96742,1868,652,6127,573,1725,724,108411,...,0.60,0.006014,5.65,0.056516,0.53,0.005285,1.59,0.015912,0.67,0.006678
3,04,alt urgell,17822,1044,212,345,150,504,100,20177,...,1.05,0.010507,1.71,0.017099,0.74,0.007434,2.50,0.024979,0.50,0.004956
4,05,alta ribagorca,3365,171,61,121,19,59,6,3802,...,1.60,0.016044,3.18,0.031825,0.50,0.004997,1.55,0.015518,0.16,0.001578


In [215]:
df_poblacio_nacionalitat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   codi                              42 non-null     object 
 1   comarca                           42 non-null     object 
 2   espanyola                         42 non-null     int64  
 3   resta_ue                          42 non-null     int64  
 4   resta_d'europa                    42 non-null     int64  
 5   africa                            42 non-null     int64  
 6   america_del_nord_i_central        42 non-null     int64  
 7   america_del_sud                   42 non-null     int64  
 8   asia_i_oceania                    42 non-null     int64  
 9   total                             42 non-null     int64  
 10  espanyola_pct                     42 non-null     float64
 11  espanyola_ratio                   42 non-null     float64
 12  resta_ue_p

In [217]:
df_poblacio_nacionalitat_clean.to_csv('clean_data/2019/df_poblacio_nacionalitat.csv', index=False)

### Dades Socioeconòmiques<a class="anchor" id="dades_socioeconomiques"></a>

#### Repetidors<a class="anchor" id="repetidors"></a>

https://educacio.gencat.cat/ca/departament/estadistiques/estadistiques-ensenyament/cursos-anteriors/curs-2021-2022/eso/

In [ ]:
df_repetidors = pd.read_csv('csv/2019/alumnes_repetidors_2019.csv', delimiter=';')

In [219]:
df_repetidors.head()

,Curs,Codi àrea territorial,Àrea territorial,Codi comarca,Comarca,Codi municipi,Municipi,Codi districte,Naturalesa,Titularitat,Estudi,Nivell,Sexe,Concert,Resultat de l'avaluació,Nombre alumnes
0,2018/2019,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PÚBLIC,DEPARTAMENT D'ENSENYAMENT,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,DONA,NaN,Alumnes avaluats,8
1,2018/2019,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PÚBLIC,DEPARTAMENT D'ENSENYAMENT,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,DONA,NaN,Alumnes avaluats,2
2,2018/2019,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PÚBLIC,DEPARTAMENT D'ENSENYAMENT,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,DONA,NaN,Alumnes avaluats,37
3,2018/2019,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PÚBLIC,DEPARTAMENT D'ENSENYAMENT,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,DONA,NaN,Alumnes avaluats,73
4,2018/2019,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PÚBLIC,DEPARTAMENT D'ENSENYAMENT,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,DONA,NaN,No promocionen,8


In [221]:
df_repetidors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25767 entries, 0 to 25766
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Curs                     25767 non-null  object
 1   Codi àrea territorial    25767 non-null  int64 
 2   Àrea territorial         25767 non-null  object
 3   Codi comarca             25767 non-null  int64 
 4   Comarca                  25767 non-null  object
 5   Codi municipi            25767 non-null  int64 
 6   Municipi                 25767 non-null  object
 7   Codi districte           25767 non-null  int64 
 8   Naturalesa               25767 non-null  object
 9   Titularitat              25767 non-null  object
 10  Estudi                   25767 non-null  object
 11  Nivell                   25767 non-null  int64 
 12  Sexe                     25767 non-null  object
 13  Concert                  9689 non-null   object
 14  Resultat de l'avaluació  25767 non-nul

In [223]:
df_repetidors['Comarca'].unique()

array(['BARCELONÈS', 'ALT EMPORDÀ', 'BAIX EMPORDÀ', 'CERDANYA',
       'GARROTXA', 'GIRONÈS', "PLA DE L'ESTANY", 'RIPOLLÈS', 'SELVA',
       'ALT URGELL', 'ALTA RIBAGORÇA', 'GARRIGUES', 'NOGUERA',
       'PALLARS JUSSÀ', 'PALLARS SOBIRÀ', "PLA D'URGELL", 'SEGARRA',
       'SEGRIÀ', 'URGELL', "VAL D'ARAN", 'ALT CAMP', 'BAIX CAMP',
       'BAIX PENEDÈS', 'CONCA DE BARBERÀ', 'PRIORAT', 'TARRAGONÈS',
       'ALT PENEDÈS', 'GARRAF', 'BAIX EBRE', 'MONTSIÀ', "RIBERA D'EBRE",
       'TERRA ALTA', 'BAIX LLOBREGAT', 'VALLÈS OCCIDENTAL', 'MARESME',
       'VALLÈS ORIENTAL', 'ANOIA', 'BAGES', 'BERGUEDÀ', 'OSONA',
       'SOLSONÈS', 'MOIANÈS'], dtype=object)

In [225]:
# Netejar el DataFrame 'df_repetidors' utilitzant la funció 'clean_dataframe'
df_repetidors_clean = clean_dataframe(df_repetidors)
df_repetidors_clean.head()

,curs,codi_area_territorial,area_territorial,codi_comarca,comarca,codi_municipi,municipi,codi_districte,naturalesa,titularitat,estudi,nivell,sexe,concert,resultat_de_l'avaluacio,nombre_alumnes
0,2018/2019,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,public,departament d'ensenyament,educacio secundaria obligatoria,1,dona,NaN,alumnes avaluats,8
1,2018/2019,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,public,departament d'ensenyament,educacio secundaria obligatoria,1,dona,NaN,alumnes avaluats,2
2,2018/2019,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,public,departament d'ensenyament,educacio secundaria obligatoria,1,dona,NaN,alumnes avaluats,37
3,2018/2019,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,public,departament d'ensenyament,educacio secundaria obligatoria,1,dona,NaN,alumnes avaluats,73
4,2018/2019,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,public,departament d'ensenyament,educacio secundaria obligatoria,1,dona,NaN,no promocionen,8


In [226]:
# Filtrar les dades del DataFrame per obtenir només les comarques rellevants
df_repetidors_clean = filtrar_comarques(df_repetidors_clean, 'comarca')

In [229]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
display(df_repetidors_clean['comarca'].unique())
print(len(df_repetidors_clean['comarca'].unique()))

array(['barcelones', 'alt emporda', 'baix emporda', 'cerdanya',
       'garrotxa', 'girones', "pla de l'estany", 'ripolles', 'selva',
       'alt urgell', 'alta ribagorca', 'garrigues', 'noguera',
       'pallars jussa', 'pallars sobira', "pla d'urgell", 'segarra',
       'segria', 'urgell', 'alt camp', 'baix camp', 'baix penedes',
       'conca de barbera', 'priorat', 'tarragones', 'alt penedes',
       'garraf', 'baix ebre', 'montsia', "ribera d'ebre", 'terra alta',
       'baix llobregat', 'valles occidental', 'maresme',
       'valles oriental', 'anoia', 'bages', 'bergueda', 'osona',
       'solsones', 'moianes'], dtype=object)

41


In [231]:
# Obtenir els valors únics de la columna 'resultat_de_l'avaluacio' en el DataFrame netejat
df_repetidors_clean['resultat_de_l\'avaluacio'].unique()

array(['alumnes avaluats', 'no promocionen', 'promocionen al mes de juny',
       'promocionen al mes de setembre', 'promocionen amb pendents',
       "s'ha expedit certificat"], dtype=object)

In [233]:
# Filtrar el DataFrame per mantenir només les files amb 'resultat_de_l'avaluacio' igual a 'no promocionen'
df_no_promocionen = df_repetidors_clean[df_repetidors_clean['resultat_de_l\'avaluacio'] == 'no promocionen']

# Agrupar les dades per 'comarca' i 'sexe', sumant els valors de 'nombre_alumnes'
df_repetidors = df_no_promocionen.groupby(['comarca', 'sexe'])['nombre_alumnes'].sum().reset_index()

# Renombrar la columna 'nombre_alumnes' a 'repetixen'
df_repetidors = df_repetidors.rename(columns={'nombre_alumnes': 'repetixen'})

# Mostrar les primeres files del DataFrame resultant
df_repetidors.head()

,comarca,sexe,repetixen
0,alt camp,dona,71
1,alt camp,home,115
2,alt emporda,dona,254
3,alt emporda,home,438
4,alt penedes,dona,169


In [235]:
# Afegim la columna 'codi'  per fer match amb els noms de les comarques
df_repetidors.insert(0, 'codi', df_repetidors['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_repetidors = df_repetidors.sort_values(by='codi').reset_index(drop=True)
df_repetidors.head()

,codi,comarca,sexe,repetixen
0,01,alt camp,dona,71
1,01,alt camp,home,115
2,02,alt emporda,dona,254
3,02,alt emporda,home,438
4,03,alt penedes,dona,169


In [237]:
df_repetidors.to_csv('clean_data/2019/df_repetidors.csv', index=False)

### Total Repetidors per Comarca<a class="anchor" id="repetidors_comarca"></a>

In [239]:
df_repetidors.head()

,codi,comarca,sexe,repetixen
0,01,alt camp,dona,71
1,01,alt camp,home,115
2,02,alt emporda,dona,254
3,02,alt emporda,home,438
4,03,alt penedes,dona,169


In [241]:
# Agrupar les dades per 'codi' i 'comarca', sumant els valors de 'repetixen'
df_total_repetidors = df_repetidors.groupby(['codi', 'comarca'])['repetixen'].sum().reset_index()

# Renombrar la columna 'repetixen' a 'total_repetixen'
df_total_repetidors.rename(columns={'repetixen': 'total_repetixen'}, inplace=True)
df_total_repetidors.head()

,codi,comarca,total_repetixen
0,01,alt camp,186
1,02,alt emporda,692
2,03,alt penedes,442
3,04,alt urgell,58
4,05,alta ribagorca,19


In [243]:
df_pct_ratio_estudiants.head()

NameError: name 'df_pct_ratio_estudiants' is not defined

In [ ]:
# Fusionar els DataFrames 'df_total_repetidors' i 'df_pct_ratio_estudiants' utilitzant la columna comuna 'codi'
df_total_repetidors_merged = df_total_repetidors.merge(df_pct_ratio_estudiants[['codi', 'poblacio_estudiant']], on='codi', how='left')

# Seleccionar només les columnes necessàries del DataFrame resultant
df_total_repetidors_merged = df_total_repetidors_merged[['codi', 'comarca', 'total_repetixen', 'poblacio_estudiant']].copy()
df_total_repetidors_merged.head()

In [ ]:
# Calcular el percentatge de repetidors respecte a la població estudiant
df_total_repetidors_merged['pct_repetidors'] = (df_total_repetidors_merged['total_repetixen'] / df_total_repetidors_merged['poblacio_estudiant'] * 100).round(2)

# Calcular la ràtio de repetidors respecte a la població estudiant
df_total_repetidors_merged['ratio_repetidors'] = (df_total_repetidors_merged['total_repetixen'] / df_total_repetidors_merged['poblacio_estudiant'])
df_total_repetidors_merged.head()

In [ ]:
df_total_repetidors_merged.to_csv('clean_data/2019/df_total_repetidors.csv', index=False)

### Repetidors Població<a class="anchor" id="repetidors_poblacio"></a>

In [ ]:
df_repetidors.head()

In [ ]:
df_total_repetidors_merged.head()

In [ ]:
# Fusionar 'df_repetidors' amb 'df_total_repetidors_merged' per afegir 'comarca' i 'poblacio_estudiant'
df_repetidors_genere_merged = df_repetidors.merge(
    df_total_repetidors_merged[['codi', 'comarca', 'poblacio_estudiant']], 
    on='codi', 
    how='left'
)
df_repetidors_genere_merged.head()

In [ ]:
# Eliminar 'comarca_y' i renombrar 'comarca_x' a 'comarca'
df_repetidors_genere_merged = df_repetidors_genere_merged.drop(columns=['comarca_y']).rename(columns={'comarca_x': 'comarca'})

# Calcular el total de repetidors i el total d'estudiants per comarca
df_total_repetidors = df_repetidors_genere_merged.groupby(['codi', 'comarca'])[['repetixen', 'poblacio_estudiant']].sum().reset_index()
df_total_repetidors.rename(columns={'repetixen': 'total_repetixen', 'poblacio_estudiant': 'total_estudiants'}, inplace=True)

# Afegir els totals al DataFrame original i mantenir 'comarca'
df_repetidors_genere_merged = df_repetidors_genere_merged.merge(
    df_total_repetidors[['codi', 'comarca', 'total_repetixen', 'total_estudiants']], 
    on=['codi', 'comarca'], 
    how='left'
)

df_repetidors_genere_merged.head()

In [ ]:
# Calcular el percentatge de repetidors per dones
df_repetidors_genere_merged['pct_repetidors_dones'] = (
    df_repetidors_genere_merged.loc[df_repetidors_genere_merged['sexe'] == 'dona', 'repetixen'] /
    df_repetidors_genere_merged['total_estudiants'] * 100
).fillna(0).round(2)

# Calcular el percentatge de repetidors per homes
df_repetidors_genere_merged['pct_repetidors_homes'] = (
    df_repetidors_genere_merged.loc[df_repetidors_genere_merged['sexe'] == 'home', 'repetixen'] /
    df_repetidors_genere_merged['total_estudiants'] * 100
).fillna(0).round(2)

# Calcular la ràtio de repetidors per dones
df_repetidors_genere_merged['ratio_repetidors_dones'] = (
    df_repetidors_genere_merged.loc[df_repetidors_genere_merged['sexe'] == 'dona', 'repetixen'] /
    df_repetidors_genere_merged['total_estudiants']
).fillna(0)

# Calcular la ràtio de repetidors per homes
df_repetidors_genere_merged['ratio_repetidors_homes'] = (
    df_repetidors_genere_merged.loc[df_repetidors_genere_merged['sexe'] == 'home', 'repetixen'] /
    df_repetidors_genere_merged['total_estudiants']
).fillna(0)

# Seleccionar només les columnes necessàries i eliminar duplicats
df_repetidors_poblacio = df_repetidors_genere_merged[['codi', 'comarca', 'total_repetixen', 
                                         'pct_repetidors_dones', 'pct_repetidors_homes', 
                                         'ratio_repetidors_dones', 'ratio_repetidors_homes']].drop_duplicates()

# Mostrar el DataFrame resultant
df_repetidors_poblacio.head()


In [ ]:
# Agrupar per comarca i codi per sumar els valors de dones i homes
df_repetidors_poblacio = df_repetidors_poblacio.groupby(['codi', 'comarca'], as_index=False).sum()

df_repetidors_poblacio.head()

In [ ]:
df_repetidors_poblacio.to_csv('clean_data/2019/df_repetidors_poblacio.csv', index=False)

### Nivell Socioeconòmic (Renda Familiar)<a class="anchor" id="renta_familiar"></a>

In [ ]:
df_renta_familiar = pd.read_csv('csv/2019/renta_familiar2019.csv', delimiter=';', skiprows=6)

In [245]:
df_renta_familiar.head()

,Unnamed: 0,RFDB (miles de euros),RFDB por habitante (euros),RFDB por habitante (Índex Catalunya=100 por habitante)
0,Alt Camp,720501,16266,"88,1"
1,Alt Empordà,2034758,14802,"80,2"
2,Alt Penedès,1839020,17023,"92,2"
3,Alt Urgell,293967,14569,"78,9"
4,Alta Ribagorça,61538,16124,"87,3"


In [247]:
# Renombrar la columna 'Unnamed: 0' a 'comarca'
df_renta_familiar = df_renta_familiar.rename(columns={'Unnamed: 0': 'comarca'})
df_renta_familiar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 4 columns):
 #   Column                                                  Non-Null Count  Dtype 
---  ------                                                  --------------  ----- 
 0   comarca                                                 51 non-null     object
 1   RFDB (miles de euros)                                   51 non-null     int64 
 2   RFDB por habitante (euros)                              51 non-null     int64 
 3   RFDB por habitante (Índex Catalunya=100 por habitante)  51 non-null     object
dtypes: int64(2), object(2)
memory usage: 1.7+ KB


In [249]:
# Netejar el DataFrame 'df_renta_familiar' utilitzant la funció 'clean_dataframe'
df_renta_familiar_clean = clean_dataframe(df_renta_familiar)
# Filtrar les dades per obtenir només les comarques rellevants
df_renta_familiar_clean = filtrar_comarques(df_renta_familiar_clean, 'comarca')
df_renta_familiar_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column                                                  Non-Null Count  Dtype 
---  ------                                                  --------------  ----- 
 0   comarca                                                 42 non-null     object
 1   rfdb_(miles_de_euros)                                   42 non-null     int64 
 2   rfdb_por_habitante_(euros)                              42 non-null     int64 
 3   rfdb_por_habitante_(index_catalunya=100_por_habitante)  42 non-null     object
dtypes: int64(2), object(2)
memory usage: 1.4+ KB


In [251]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_renta_familiar_clean.insert(0, 'codi', df_renta_familiar_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_renta_familiar_clean = df_renta_familiar_clean.sort_values(by='codi').reset_index(drop=True)
df_renta_familiar_clean.head()

,codi,comarca,rfdb_(miles_de_euros),rfdb_por_habitante_(euros),rfdb_por_habitante_(index_catalunya=100_por_habitante)
0,01,alt camp,720501,16266,"88,1"
1,02,alt emporda,2034758,14802,"80,2"
2,03,alt penedes,1839020,17023,"92,2"
3,04,alt urgell,293967,14569,"78,9"
4,05,alta ribagorca,61538,16124,"87,3"


In [253]:
# Convertir els valors de la columna a format numèric
df_renta_familiar_clean['rfdb_por_habitante_(index_catalunya=100_por_habitante)'] = (
    df_renta_familiar_clean['rfdb_por_habitante_(index_catalunya=100_por_habitante)']
    .str.replace(',', '.')  # Substituir les comes per punts
    .astype(float)          # Convertir els valors a tipus float
)
df_renta_familiar_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   codi                                                    42 non-null     object 
 1   comarca                                                 42 non-null     object 
 2   rfdb_(miles_de_euros)                                   42 non-null     int64  
 3   rfdb_por_habitante_(euros)                              42 non-null     int64  
 4   rfdb_por_habitante_(index_catalunya=100_por_habitante)  42 non-null     float64
dtypes: float64(1), int64(2), object(2)
memory usage: 1.8+ KB


In [255]:
df_renta_familiar_clean.head()

,codi,comarca,rfdb_(miles_de_euros),rfdb_por_habitante_(euros),rfdb_por_habitante_(index_catalunya=100_por_habitante)
0,01,alt camp,720501,16266,88.1
1,02,alt emporda,2034758,14802,80.2
2,03,alt penedes,1839020,17023,92.2
3,04,alt urgell,293967,14569,78.9
4,05,alta ribagorca,61538,16124,87.3


In [257]:
df_renta_familiar_clean.to_csv('clean_data/2019/df_renta_familiar.csv', index=False)

### Nivell Socioeconòmic (RFBD)<a class="anchor" id="rfbd"></a>

In [ ]:
df_rfbd = pd.read_csv('csv/2019/RFBD_2019.csv', delimiter=';', skiprows=6)

In [259]:
df_rfbd.head()

,Unnamed: 0,Recursos. Remuneración asalariados,Recursos. Excedente bruto explotación,Recursos. Prestaciones sociales,Usos. Cotizaciones sociales,Usos. Impuestos
0,Alt Camp,565929,186515,206768,171532,91718
1,Alt Empordà,1551611,680144,484650,486006,309468
2,Alt Penedès,1527182,496000,510175,452437,259924
3,Alt Urgell,187736,95819,85046,60167,34504
4,Alta Ribagorça,41587,24789,15395,13962,7562


In [261]:
# Renombrar la columna 'Unnamed: 0' a 'comarca'
df_rfbd = df_rfbd.rename(columns={'Unnamed: 0': 'comarca'})
df_rfbd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 6 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   comarca                                51 non-null     object
 1   Recursos. Remuneración asalariados     51 non-null     int64 
 2   Recursos. Excedente bruto explotación  51 non-null     int64 
 3   Recursos. Prestaciones sociales        51 non-null     int64 
 4   Usos. Cotizaciones sociales            51 non-null     int64 
 5   Usos. Impuestos                        51 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 2.5+ KB


In [263]:
# Netejar el DataFrame 'df_renta_familiar' utilitzant la funció 'clean_dataframe'
df_rfbd_clean = clean_dataframe(df_rfbd)
# Filtrar les dades per obtenir només les comarques rellevants
df_rfbd_clean = filtrar_comarques(df_rfbd_clean, 'comarca')
df_rfbd_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 6 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   comarca                                42 non-null     object
 1   recursos._remuneracion_asalariados     42 non-null     int64 
 2   recursos._excedente_bruto_explotacion  42 non-null     int64 
 3   recursos._prestaciones_sociales        42 non-null     int64 
 4   usos._cotizaciones_sociales            42 non-null     int64 
 5   usos._impuestos                        42 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 2.1+ KB


In [265]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_rfbd_clean.insert(0, 'codi', df_rfbd_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_rfbd_clean = df_rfbd_clean.sort_values(by='codi').reset_index(drop=True)
df_rfbd_clean.head()

,codi,comarca,recursos._remuneracion_asalariados,recursos._excedente_bruto_explotacion,recursos._prestaciones_sociales,usos._cotizaciones_sociales,usos._impuestos
0,01,alt camp,565929,186515,206768,171532,91718
1,02,alt emporda,1551611,680144,484650,486006,309468
2,03,alt penedes,1527182,496000,510175,452437,259924
3,04,alt urgell,187736,95819,85046,60167,34504
4,05,alta ribagorca,41587,24789,15395,13962,7562


In [267]:
df_rfbd_clean.to_csv('clean_data/2019/df_rfbd.csv', index=False)